# Periodic Electronic Structure: Coefficient Generation

<font size="3">

<hr>
DISTRIBUTION STATEMENT A. Approved for public release. Distribution is unlimited.

This material is based upon work supported by the Under Secretary of Defense for
Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions,
findings, conclusions or recommendations expressed in this material are those of the
author(s) and do not necessarily reflect the views of the Under Secretary of Defense
for Research and Engineering.

© 2023 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part
252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government
rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed
above. Use of this work other than as specifically authorized by the U.S. Government
may violate any copyrights that exist in this work.
<hr><hr>
</font>

In [1]:
pest_path="../../../src/PEST"

push!(LOAD_PATH,pest_path)

# using Pkg
# Pkg.develop(pest_path)

using  PEST

# Overview

In this notebook, we detail how to use `PEST`, a Julia package, to generate periodic electronic structure coefficients. This includes tuning parameters which control the system size in terms of the number of basis elements. For details on general `PEST` usage, see [Examples/PEST](../../PEST/).

# Problem description

Here we will focus on periodic electronic structure systems expressed in the dual plane-wave (DPW) basis. The Hamiltonians for this class of problems take on the form
$$
H = \sum_{p,q,\sigma} T(p-q) a^\dagger_{p,\sigma}a_{q,\sigma} + \sum_{p,\sigma} U(p) n_{p,\sigma} + \sum_{(p,\alpha)\neq(q,\beta)} V(p-q)n_{p,\alpha}n_{q,\beta}
$$

where $a^\dagger_{p,\sigma}$ and $a_{p,\sigma}$ are fermionic creation and annihilation operators on spatial orbital $p$ with spin $\sigma \in \{\uparrow,\downarrow\}$, and $n_{p,\sigma} = a^\dagger_{p,\sigma}a_{p,\sigma}$ is the number operator. The coefficients are expressed in the DPW basis as
\begin{align*}
T(p-q) &= \frac{1}{2N_{so}} \sum_\nu k_\nu^2 \, \cos \big[ k_\nu \cdot r_{p-q} \big]\\
U(p) &= -\frac{4\pi}{\Omega} \sum_{j,\nu \neq 0 }  \frac{\zeta_j \, \cos \big[ k_\nu \cdot (R_j - r_p)\big]}{k_\nu^2}\\
V(p-q) &= \frac{2\pi}{\Omega} \sum_{\nu \neq 0} \frac{\cos \big[k_\nu \cdot r_{p-q} \big]}{k_\nu^2},
\end{align*}
where each spatial orbital $p$ is associated with an orbital centroid $r_p = p(2\Omega/N_{so})^{1/3}$, $\Omega$ is the computational cell volume, $R_j$ is the position of atom $j$ with atomic number $\zeta_j$, and $N_{so}$ is the number of spin-orbitals. The momentum modes are defined as $k_\nu = 2\pi\nu / \Omega^{1/3}$ with $\nu \in [-(N_{so}/2)^{1/3}, (N_{so}/2)^{1/3}]^{\otimes 3}$.

Under the Jordan-Wigner transformation, this becomes

$$
H = \sum_{p\neq q,\sigma} \frac{T(p-q)}{2} \left( X_{p,\sigma} \vec{Z} X_{q,\sigma} + Y_{p,\sigma} \vec{Z} Y_{q,\sigma}\right) + \sum_{(p,\alpha)\neq(q,\beta)} \frac{V(p-q)}{4} Z_{p,\alpha}Z_{q,\beta} \\
- \sum_{p,\sigma} \left( \frac{T(0)+U(p)+\sum_q V(p-q)}{2} \right) Z_{p,\sigma} + \sum_{p} \left( T(0)+U(p)+\sum_q \frac{V(p-q)}{2} \right) \mathbb{1}
$$

where the notation $A_j \vec{Z} A_k$ represents the operator $A_j Z_{j+1}\dots Z_{k-1} A_k$. Note, the index $p$ is a $d$-dimensional vector with components $p_i \in [0,M_i-1]$, such that the total number of spin-orbitals is $N_{so}=2\prod_{i=0}^{d-1} M_i$. This vector index is mapped onto an integer using the mapping function
$$
f(p) = \sum_{i=0}^{d-1}p_i (M_i)^i
$$

# Problem set up: from material geometry to Hamiltonian coefficients using PEST

The starting point for our example is the description of the geometry of a given material. A few examples are listed below and can be selected by changing the choice of `example_key`. To generate the coefficients for each example using `PEST`, we need a set of coordinates that describe the atoms in a "supercell" and a vector containing the vacuum padding between periodic images (see Figure below). Here, we choose an orthorhombic supercell for which the atomic coordinates, expressed in units of Angstrom, are contained in an xyz file. For the padding, we use a uniform 10 Angstroms for the sake of simplicity. See [pest-intro](../../PEST/pest-intro.ipynb) for details on how to determine the padding based on physical considerations.

<center>
<img src="./figures/figure-slab.png" width=800>
</center>

In [2]:
# list of available examples
examples = ["magnesium-4x4x2","cyclic-ozone","ozone","magnesium-8x8x2"]
# associated filenames
filenames = ["magnesium-4x4x2-orthorhombic.xyz","c60-o3-opt.xyz","o3-opt.xyz","magnesium-8x8x2-orthorhombic.xyz"]
# dictionary for easy access
dict_of_usable_geometries = Dict(zip(examples,filenames))
# filepath to geometries
filepath = "../../../data/geometries/";

In [3]:
# example selection: change example_key to switch to a different system
example_key = examples[1]
println("Selected example: $example_key")
example_geometry_filename = filepath * dict_of_usable_geometries[example_key];

Selected example: magnesium-4x4x2


The size of the problem basis, $N=N_{so}/2$, is parameterized by the converged plane-wave energy cutoff, `E_cut_off`, in units of Rydberg. For a small, simple example, we define the cut off to be unphysically low, but the parameter can be varied to analyze resource estimates as a function of basis size.

Once the geometry is chosen and the cut off is set, we can call `calc_hamiltonian`, which returns the calculated Hamiltonian coefficients and the associated real space grid mesh. We can see details of the grid based on our chosen parameters in the print out. In particular, note the line `Grid = ...` which tells us the number of basis elements along each dimension and the resulting total number of basis elements. Note, for later block encodings, it is important that there are 3 or more elements along each dimension, ie `Grid = 3 x 3 x 3`, and that the grid is a cube. These constraints can be met by changing `E_cut_off`. Increasing `E_cut_off` will increase the overall basis size, and decreasing it will decrease the basis size.

In [4]:
# Plane-wave energy cutoff. Increase to increase number of basis functions.
E_cut_off = .12

ham,grid = calc_hamiltonian( example_geometry_filename,
                            pad=[10.0,10.0,10.0],
                            E_cut = E_cut_off );

###  Requested a DPW Hamiltonian.

	Cutoff Equiv.  =  0.12 Ry 
	Grid Spacing   =  18.137994 Bohr   (scaling = 1.0)


	Grid           =  3 x 3 x 3  ( 27 basis functions ).

###  Loaded geometry for 64 atoms.

	Lattice Dimensons  =  43.161 x 39.91 x 38.594 (Bohr)


	Actual Spacing     =  14.387 x 13.303 x 12.865 (Bohr)

###  Building grid and mesh.



	Building grid distance permutations: 


	Applying symmetry operations: 


	Generated a minimal set of 63 displacement vectors.


	Generated a minimal set of 10 distances.

###  Calculating the Hamiltonian.

	Calculating Kinetic & E-E Interaction Terms:


	Calculating E-Ion Terms:


Finally, we write these to hdf5 files, which will facilitate later interfacing with python. In particular, these files can be input to a pyLIQTR ElectronicStructure problem instance class, which provides easy access to pyLIQTR block encoding implementations, as well as phase estimation and qubitized dynamics algorithms. See [problem_instance_and_encoding](problem_instance_and_encoding.ipynb) for details.

In [5]:
write_hamiltonian_hdf5("example.ham.hdf5",ham);
write_grid_hdf5("example.grid.hdf5",grid);